# LLM RAG

<https://arxiv.org/abs/2312.10997>

《Retrieval-Augmented Generation for Large Language Models: A Survey》这篇论文介绍了 RAG 技术的发展。

本文尝试对其关键要点进行一些简单的介绍。

## Why Need RAG, LLM 面临的问题

在介绍 RAG 是什么以前，先介绍了 LLM 目前所面临的问题

### 训练成本

神经网络类型的 AI 有一大特点，就是预训练的成本远远高于推理。

OpenAI 在 Dev Day 2023 宣称 GPT 的预训练成本在 2～3 百万美元。

### 数据集落后

高昂的预训练成本带来一个最直接的问题就是：模型更新缓慢。 GPT-3.5 的数据集时间为 September 2021。

而且数据集的更新成本不仅仅是训练，还有数据集的收集和清洗，这都进一步降低了模型更新的频率。

### hallucination 幻觉

有一种观点认为 LLM 是一种高效的有损信息压缩算法，它的信息解压缩过程依赖于用户 prompt。

这使得 LLM 的答复质量非常容易受到用户 prompt 的干扰。

![](https://s3.laisky.com/uploads/2023/12/llm-hallucination.png)

### Transparency 透明性

LLM 给出的回答完全是黑盒，根本不知道来自哪，自然也就难以查证。

## What is RAG

### Parameter

我们将 LLM 的信息分为两个渠道：

1. Parameteric knowledge：预训练 LLM 时使用的信息
2. Non-parametric knowledge：LLM 推理时 context 内的信息

### RAG

根据 LLM 对 parameter 的依赖程度，可以再分为三类：

1. fully parameterized model：只依赖预训练数据
2. RAG：hybrid
3. RCG：完全依赖推理时的外部信息

所以可以对 RAG 进行定义：Retrieval-Augmented Generation，基于信息抓取的生成。

或者更通俗的理解为：在直接将数据交给模型以前，先进行一轮信息检索，完善输入信息。

### RAG Vs. Fine-Tunning

RAG 和 fine-tuning 都是可以提高 LLM 模型性能的方法，两者的应用场景存在一些差别：

1. RAG 适合少量垂直领域的精确信息，不适合开放式的大量数据
2. fine-tuning 适合大量数据

需要注意的是，fine-tuning 和 pre-traning 的区别在于，fine-tuning 不适合让 LLM 学习新知识，而是适合让 LLM 强化某个已知知识。

可以认为 fine-tuning 是复习，RAG 是考试时的小抄。

#### What is fine-tuning

其实就是组织大量结构化的问答数据喂给 LLM，增强其对特定语料的回答能力。

![](https://s3.laisky.com/uploads/2023/12/llm-fine-tunning-sample.png)

fine-tuning 和 RAG 并不矛盾，两者契合可以发挥出更大的作用

![](https://s3.laisky.com/uploads/2023/12/llm-rag-finetuning.png)

![](https://s3.laisky.com/uploads/2023/12/llm-rag-vs-finetuning.png)

## RAG's Evolution

介绍 RAG 技术和工作流的进化

### Naive RAG

最简单的 RAG，就是顾名思义的执行三个步骤：

1. Retrieval: 根据 prompt 抓取外部数据
2. Augmented: 使用外部数据增强 prompt 
3. Generation: 把增强后的 prompt 交给 LLM，生成 predict/answer

#### embeddings

Naive 处理资料集的方式也是单一的：

1. 首先，全部转换为文字（text）
2. 对文字进行切块（chunk）
3. 把每一个 chunk 交给 embeddings-model，计算词向量（word vector）
4. 将词向量和 chunk 存储向量数据库

embeddings 就是基于 LLM 将一个语句转化为一个高维向量，切分资料的 `chunk_size` 是一个关键参数。

![](https://s3.laisky.com/uploads/2023/12/llm-embaeedings-sample.png)

#### Retrieval

执行类似步骤：

1. 将 prompt 交给 embeddings-model，计算词向量
2. 在向量数据库中执行 KNN 查询，获取 K 个最近似结果


#### Augmented

将获取到的资料按照某个模板，和 prompt 进行拼接，得到最终 prompt

#### Generation

将最终 prompt 交给 LLM，得到回答。

一个简单的结合向量数据库实现 prompt 增强的例子：

![](https://s3.laisky.com/uploads/2023/12/llm-embeddings-code.png)

#### 向量化的问题

可以看出数据的检索完全依赖于向量化所产生的词向量。

> when an embedding model calculates the vector representation of a sentence, it does so based on the similarity of the sentence to the pre-trained data

而 embedding model 在计算一个语句的向量时，需要基于 pre-training 的数据来理解输入的语句。如果输入的语句和预训练数据集的差异特别大，会导致结果出现很大偏差。

基于词向量相似性搜索的方案，precision 和 recall 都很低。就是查询的数据不一定有用，有用的数据不一定被查询到。

Ps. precision 度量结果数据中的阳性率。recall 度量所有阳性被找到的概率。都是越高越好。

### Advanced RAG

Advanced RAG 在 Naive 的基础上增加了 pre-retrieval 和 post-retrieval 两个方法。

工作流变为：

1. pre-retrieval process
2. embeddings
3. post-retrieval process

#### Pre-retrieval Process

在 embeddings 以前，对数据进行清洗和规整，可以分为 5 个步骤：

1. Enhancing Data Granularity: 对数据内容进行修订和简化，确保数据源的正确性和可读性
2. Optimizing Index Structures: 优化数据索引，引入图数据库等关联结构
3. Adding Medadata Information: 为切块后的数据增加 metadata，标记数据来源
4. Alignment Optimization: 可以为每一个 chunk 生成一个假设性提问，然后将这个问题本身也嵌合到 chunk 中，这样可以提高检索的关联度。
5. Mixed Retrieval: 混合使用多种检索技术，而不仅仅是词向量搜索。

#### Embedding

对 embeddings 过程中所使用的 embedding-model 也进行改进

1. Fine-tunning Embedding: 可以将领域知识预先通过 fine-tuning 内嵌到模型中
2. Dynamic Embedding: 在 embeddings 时，不要仅针对关键词（static），而是要联合上下文一起（dynamic）。

#### Post-Retrieval Process

在完成资料查询，提交给 LLM 前，继续对收集到的资料进行优化

1. ReRank: 根据关联度进行打分和重排序
2. Prompt Compression: 无关输入对 LLM 的性能有负面影响。压缩不相关信息，强调关键信息，减少总长度

#### RAG Pipeline Optimization

一些通用的 RAG 检索资料优化办法

1. Hybrid Search: 前面提到过的混合检索
2. Recursive Retrieval And Query Engine: 多阶段检索，先检索一批小 chunk，再根据小 chunk 去检索大 chunk
3. StepBack-prompt: 一种 prompt-engineering，可以显著提高推理密集型任务的性能。让 LLM 更关注抽象概念
4. Subqueries: 根据语意拆分为多个小查询
5. HyDE: 先让 LLM 回答一次，然后根据 LLM 的回答再去搜索相关资料。但如果 LLM 对相关话题不熟悉，反而会加重幻觉。

### Modular RAG

这是作者提到的 RAG 的最终进化形态。不过其实所使用的技术都是前面 naive 和 advanced 里提到过的。

最重要的改变更多是架构设计上的，将单一的命令式流水线（pipeline），变成了相应式的动态调度。

所有前面提到过的功能都被封装为了功能模块，根据任务类型进行动态组合和调度。

![](https://s3.laisky.com/uploads/2023/12/llm-modular-rag.png)

## Retriever

在 Modular RAG 中，Retriever 负责对外部数据源进行预处理和查询。

### 增强 Embeddings 的语意准确度

#### Chunk

RAG 的最终生成物不能超出 LLM 的 context window，所以数据切片是必要的，而切片的 chunk size 的选择是优化的第一步。

OpenAI 的 text-embedding-ada-002 的最优 chunk size 为 256～512 tokens。


#### Fine-tuning Embedding Models

embedding 是 RAG 的核心，为了让 embedding model 能够更好地理解垂直领域信息，可以对 embedding model 进行 fine-tuning。

Ps. OpenAI 目前尚不支持该功能

![](https://s3.laisky.com/uploads/2023/12/llm-embedding-code-2.png)

### 将用户请求和数据集进行对齐

#### Query Rewrite

最简单直观的方法就是让 LLM 重写用户查询，比如将其拆分为多个子查询。

#### Query Embedding Transformation

query rewrite 是粗粒度的，query 的 embedding 应该是细粒度的。

query 最终也需要被 embedding 然后再去搜索外部资料，处理 query 的 embedding model 也可以被 fine-tuning，以使其可以更好的匹配特定任务，尤其是使其可以更好的关联到结构化的数据

### Aligning Retriever's Output and LLM's Preference

单纯的计算 retriever 的 hit rate（正确性）是不够的，因为可能查找的资料并不是 LLM 所需要的。
相比单纯的信息正确，LLM 更偏好于可读性更好的资料。

所以 retriever 还需要对齐（alignment），才能真正提高 RAG 的性能。

### Retrieval Metrics

* `Hit Rate (HR)`: 检索到相关资料的概率
* `Recall`: 所有应该被检索的文档里，被正确检索出来的比例
* `Precesion`: 检索出来的文档里，相关文档的比率
* `Mean Reciprocal Rank (MRR)`: rerank 的指标，度量 retriever 返回的最优信息是否出现在了最前列
* `Mean Average Precision (mAP)`: 也称为 `mAP@K`，度量第 K 个正确答案的位置。可以理解为 MRR 的复数版。
* `Normalized Discounted Cumulative Gain (NDCG)`: 度量整体的 rerank 质量
* `Exact Match (EM)`: 查询到的资料里，包含正确答案的概率
* `F1 Score`: 就是 recall 和 precision 的调和平均数：
* `Semantic Answer Similarity (SAS)`: 比较正确答案和 LLM 回复（predict）间的语意相似度。

<https://laisky.notion.site/Metrics-for-Information-Retrieval-and-Question-Answering-Pipelines-d5d4e3beb820419ca494596e319befcf?pvs=4>

## Generator